In [6]:
# 한글 폰트 설치
!apt-get update -qq
!apt-get install -y fonts-nanum
# 폰트 캐시 갱신
import matplotlib.pyplot as plt
from matplotlib import font_manager, rc
import os
# 폰트 경로 확인
font_path = '/usr/share/fonts/truetype/nanum/NanumGothic.ttf'
# 시스템에 폰트를 추가
font_manager.fontManager.addfont(font_path)
rc('font', family='NanumGothic')








W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  fonts-nanum
0 upgraded, 1 newly installed, 0 to remove and 34 not upgraded.
Need to get 10.3 MB of archives.
After this operation, 34.1 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 fonts-nanum all 20200506-1 [10.3 MB]
Fetched 10.3 MB in 2s (4,205 kB/s)
Selecting previously unselected package fonts-nanum.
(Reading database ... 126102 files and directories currently installed.)
Preparing to unpack .../fonts-nanum_20200506-1_all.deb ...
Unpacking fonts-nanum (20200506-1) ...
Setting up fonts-nanum (20200506-1) ...
Processing triggers for fontconfig (2.13.1-4.2ubuntu5) ...


In [4]:
import pandas as pd
import numpy as np
import joblib
from sklearn.preprocessing import StandardScaler
from sklearn.impute import KNNImputer
from sklearn.metrics import pairwise_distances
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")

# --- Define Feature Lists (Fallbacks) ---
HE_DM_HbA1c_input_features = [
    'HE_glu', 'HE_HbA1c', 'DE1_dg', 'age', 'DI1_dg', 'HE_wc', 'DI2_dg', 'HE_sbp2', 'HE_sbp1', 'HE_BMI',
    'HE_BUN', 'HE_alt', 'HE_TG', 'HE_ast', 'L_OUT_FQ', 'HE_wt', 'D_1_1', 'HE_DMfh2', 'HE_dbp1', 'HE_dbp2',
    'BO1', 'HE_crea', 'sex', 'BP16_2', 'genertn', 'ho_incm', 'cfam', 'LS_VEG1', 'HE_chol', 'HE_LDL_drct',
    'L_BR_FQ', 'HE_Ucrea', 'HE_HDL_st2', 'edu', 'DI2_ag', 'DI2_pt', 'DI2_pr', 'DI2_2', 'DI1_ag', 'DI1_pt',
    'DI1_pr', 'DI1_2', 'DE1_33', 'DE1_31', 'DE1_32', 'DE1_34', 'DE1_ag', 'DE1_4', 'DE1_3', 'DE1_pt', 'DE1_pr'
]

HE_obe_input_features = [
    'HE_BMI', 'HE_wc', 'HE_wt', 'BO1', 'HE_alt', 'HE_TG', 'HE_sbp2', 'HE_sbp1', 'HE_glu', 'HE_dbp1',
    'HE_HbA1c', 'HE_dbp2', 'BO1_1', 'DI1_dg', 'HE_ast', 'HE_crea', 'DI2_dg', 'DI2_pt', 'DI2_pr', 'DI2_2',
    'sex', 'DI2_ag', 'DI1_pr', 'DI1_pt', 'DI1_2', 'DI1_ag', 'BO2_1', 'HE_HDL_st2'
]

HE_HP_input_features = [
    'DI1_dg', 'HE_sbp2', 'HE_sbp1', 'age', 'HE_dbp1', 'HE_dbp2', 'HE_wc', 'HE_HbA1c', 'HE_glu', 'DI2_dg',
    'HE_BMI', 'HE_ast', 'DE1_dg', 'HE_BUN', 'HE_TG', 'HE_alt', 'L_OUT_FQ', 'HE_wt', 'D_1_1', 'HE_HPfh2',
    'BD2_1', 'HE_crea', 'BD1_11', 'BP1', 'BO1', 'HE_ht', 'BD1', 'sex', 'HE_chol', 'BP16_2', 'HE_LDL_drct',
    'HE_HDL_st2', 'genertn', 'cfam', 'LS_VEG1', 'HE_Ucrea', 'DE1_ag', 'DE1_32', 'DE1_34', 'DE1_31', 'DE1_33',
    'L_BR_FQ', 'DE1_4', 'DE1_pt', 'DE1_pr', 'DE1_3', 'ho_incm', 'DI2_ag', 'DI2_pt', 'DI2_pr', 'DI2_2', 'edu',
    'DI1_ag', 'DI1_pt', 'DI1_pr', 'DI1_2'
]

kmeans_features = [
    'HE_sbp1', 'HE_dbp1', 'HE_sbp2', 'HE_dbp2', 'HE_wc', 'HE_wt', 'HE_ht', 'HE_BMI',
    'HE_glu', 'HE_HbA1c', 'HE_chol', 'HE_HDL_st2', 'HE_TG', 'HE_LDL_drct', 'HE_BUN',
    'HE_crea', 'HE_Ucrea', 'HE_ast', 'HE_alt', 'BD1_11', 'BP16_1', 'BS3_2', 'BE5_1',
    'L_OUT_FQ', 'LS_VEG1', 'LS_FRUIT'
]

# --- Load Models and Preprocessing Objects ---
def load_model_and_preprocessing(model_name, feature_list_fallback):
    try:
        model = joblib.load(f"{model_name}_decision_tree_model.pkl")
        imputer = joblib.load(f"{model_name}_imputer.pkl")
        scaler = joblib.load(f"{model_name}_scaler.pkl")
        try:
            features = joblib.load(f"{model_name}_features_list.pkl")
        except FileNotFoundError:
            print(f"Warning: {model_name}_features_list.pkl not found. Using fallback feature list.")
            features = feature_list_fallback
        print(f"{model_name.capitalize()} model loaded ({len(features)} features).")
        return model, imputer, scaler, features
    except FileNotFoundError as e:
        print(f"Error loading {model_name} model or preprocessing objects: {e}")
        exit()

# Load disease prediction models
model_diabetes, imputer_diabetes, scaler_diabetes, features_diabetes = load_model_and_preprocessing("diabetes", HE_DM_HbA1c_input_features)
model_hypertension, imputer_hypertension, scaler_hypertension, features_hypertension = load_model_and_preprocessing("hypertension", HE_HP_input_features)
model_obesity, imputer_obesity, scaler_obesity, features_obesity = load_model_and_preprocessing("obesity", HE_obe_input_features)

# --- Load Hypertension KMeans Models by HE_HP ---
def load_hypertension_kmeans(he_hp_value):
    try:
        kmeans = joblib.load(f"hypertension_kmeans_model_HE_HP_{he_hp_value}.pkl")
        imputer = joblib.load(f"hypertension_kmeans_imputer_HE_HP_{he_hp_value}.pkl")
        scaler = joblib.load(f"hypertension_kmeans_scaler_HE_HP_{he_hp_value}.pkl")
        features = joblib.load(f"hypertension_kmeans_features_list_HE_HP_{he_hp_value}.pkl")
        print(f"Hypertension KMeans model for HE_HP={he_hp_value} loaded ({len(features)} features).")
        return kmeans, imputer, scaler, features
    except FileNotFoundError as e:
        print(f"Error loading hypertension KMeans model for HE_HP={he_hp_value}: {e}")
        return None, None, None, None

# --- Load Dataset for Percentile Calculation ---
try:
    data = pd.read_csv("선택컬럼_건강데이터.csv")
except FileNotFoundError as e:
    print(f"Error loading dataset: {e}")
    exit()

# --- Cluster Descriptions (Customized for HE_HP) ---
# HE_HP 값과 클러스터 번호에 따라 다른 설명 제공 (예시)
cluster_descriptions_hypertension = {
    1: {  # HE_HP=1
        0: "Cluster 0 (HE_HP=1): Low blood pressure, moderate BMI, healthy metabolic profile.",
        1: "Cluster 1 (HE_HP=1): Slightly elevated SBP/DBP, higher TG levels.",
        2: "Cluster 2 (HE_HP=1): Moderate risk with elevated cholesterol and BMI."
    },
    2: {  # HE_HP=2
        0: "Cluster 0 (HE_HP=2): Pre-hypertension, high BMI, low HDL.",
        1: "Cluster 1 (HE_HP=2): Elevated SBP, moderate metabolic risk.",
        2: "Cluster 2 (HE_HP=2): High TG and cholesterol, active lifestyle.",
        3: "Cluster 3 (HE_HP=2): High risk with elevated liver markers."
    },
    3: {  # HE_HP=3
        0: "Cluster 0 (HE_HP=3): Stage 1 hypertension, high BMI, high TG.",
        1: "Cluster 1 (HE_HP=3): Elevated SBP/DBP, moderate cholesterol.",
        2: "Cluster 2 (HE_HP=3): High risk with metabolic syndrome traits."
    },
    4: {  # HE_HP=4
        0: "Cluster 0 (HE_HP=4): Stage 2 hypertension, very high BMI, low HDL.",
        1: "Cluster 1 (HE_HP=4): Severe SBP/DBP elevation, high cholesterol.",
        2: "Cluster 2 (HE_HP=4): High TG, elevated liver markers.",
        3: "Cluster 3 (HE_HP=4): Extreme risk with multiple comorbidities."
    }
}

# --- User Input ---
user_data = {
    'HE_glu': 110.5, 'HE_HbA1c': 6.1, 'DE1_dg': 0, 'age': 55, 'DI1_dg': 0, 'HE_wc': 90.2,
    'DI2_dg': 0, 'HE_sbp2': 135, 'HE_sbp1': 138, 'HE_BMI': 26.5, 'HE_BUN': 18.0, 'HE_alt': 30.0,
    'HE_TG': 160, 'HE_ast': 25.0, 'L_OUT_FQ': 3, 'HE_wt': 78.5, 'D_1_1': 0, 'HE_DMfh2': 0,
    'HE_dbp1': 85, 'HE_dbp2': 83, 'BO1': 1, 'HE_crea': 1.1, 'sex': 1, 'BP16_2': 5,
    'genertn': 4, 'ho_incm': 5, 'cfam': 3, 'LS_VEG1': 2, 'HE_chol': 220, 'HE_LDL_drct': 140,
    'L_BR_FQ': 1, 'HE_Ucrea': 80.0, 'HE_HDL_st2': 45.0, 'edu': 3, 'DI2_ag': 0, 'DI2_pt': 0,
    'DI2_pr': 0, 'DI2_2': 0, 'DI1_ag': 0, 'DI1_pt': 0, 'DI1_pr': 0, 'DI1_2': 0,
    'DE1_33': 0, 'DE1_31': 0, 'DE1_32': 0, 'DE1_34': 0, 'DE1_ag': 0, 'DE1_4': 0, 'DE1_3': 0,
    'DE1_pt': 0, 'DE1_pr': 0, 'HE_HPfh2': 0, 'BD2_1': 0, 'BD1_11': 2, 'BP1': 0, 'HE_ht': 170,
    'BD1': 0, 'BO1_1': 0, 'BO2_1': 0, 'BP16_1': 4, 'BS3_2': 10, 'BE5_1': 2, 'LS_FRUIT': 3,
    'HE_HP': 2  # 예시로 HE_HP=2 설정
}

# --- Determine HE_HP for KMeans ---
# 실제로는 고혈압 모델 예측 결과로 HE_HP를 결정할 수 있음
user_he_hp = user_data.get('HE_HP', 1)  # 기본값 HE_HP=1
if user_he_hp not in [1, 2, 3, 4]:
    print(f"Invalid HE_HP value: {user_he_hp}. Defaulting to HE_HP=1.")
    user_he_hp = 1

# Load hypertension KMeans model
kmeans_hypertension, imputer_hypertension_kmeans, scaler_hypertension_kmeans, features_hypertension_kmeans = load_hypertension_kmeans(user_he_hp)

# --- Process User Input ---
def process_user_input(user_data, features, imputer, scaler):
    try:
        user_df = pd.DataFrame([user_data]).reindex(columns=features)
        user_imputed = pd.DataFrame(imputer.transform(user_df), columns=features)
        user_scaled = scaler.transform(user_imputed)
        return user_df, user_scaled
    except Exception as e:
        print(f"Error processing user input: {e}")
        return None, None

# Process inputs for disease prediction
user_df_diabetes, user_scaled_diabetes = process_user_input(user_data, features_diabetes, imputer_diabetes, scaler_diabetes)
user_df_hypertension, user_scaled_hypertension = process_user_input(user_data, features_hypertension, imputer_hypertension, scaler_hypertension)
user_df_obesity, user_scaled_obesity = process_user_input(user_data, features_obesity, imputer_obesity, scaler_obesity)

# Process input for hypertension KMeans
user_df_hypertension_kmeans, user_scaled_hypertension_kmeans = process_user_input(user_data, features_hypertension_kmeans, imputer_hypertension_kmeans, scaler_hypertension_kmeans)

# --- Disease Prediction ---
def predict_disease_probabilities(user_scaled, model, stages):
    try:
        if user_scaled is None:
            raise ValueError("Invalid input data for prediction.")
        probs = model.predict_proba(user_scaled)[0]
        stage_index = np.argmax(probs)
        stage = stages.get(stage_index, "Unknown")
        prob = np.max(probs) * 100
        return {"stage": stage, "probability": prob}
    except Exception as e:
        print(f"Error in prediction: {e}")
        return {"stage": "Error", "probability": 0.0}

# Define stage mappings
diabetes_stages = {0: "Normal", 1: "Pre-diabetes", 2: "Diabetes"}
hypertension_stages = {0: "Normal", 1: "Pre-hypertension", 2: "Stage 1 Hypertension", 3: "Stage 2 Hypertension"}
obesity_stages = {0: "Normal", 1: "Overweight", 2: "Obese", 3: "Severely Obese"}

# Predict diseases
disease_results = {
    "diabetes": predict_disease_probabilities(user_scaled_diabetes, model_diabetes, diabetes_stages),
    "hypertension": predict_disease_probabilities(user_scaled_hypertension, model_hypertension, hypertension_stages),
    "obesity": predict_disease_probabilities(user_scaled_obesity, model_obesity, obesity_stages)
}

# --- Cluster Assignment ---
def assign_cluster(user_scaled, kmeans, cluster_descriptions, he_hp=None):
    try:
        if user_scaled is None:
            raise ValueError("Invalid input data for clustering.")
        cluster_label = kmeans.predict(user_scaled)[0]
        if he_hp is not None:
            cluster_description = cluster_descriptions.get(he_hp, {}).get(cluster_label, "No description available.")
        else:
            cluster_description = cluster_descriptions.get(cluster_label, "No description available.")
        return cluster_label, cluster_description
    except Exception as e:
        print(f"Error in cluster assignment: {e}")
        return None, "Cluster assignment failed."

# Hypertension KMeans cluster assignment
if kmeans_hypertension is not None:
    cluster_label_hypertension, cluster_description_hypertension = assign_cluster(
        user_scaled_hypertension_kmeans, kmeans_hypertension, cluster_descriptions_hypertension, user_he_hp
    )
else:
    cluster_label_hypertension, cluster_description_hypertension = None, "Hypertension KMeans model not loaded."

# --- Percentile Calculation and Visualization ---
def calculate_percentile_and_visualize(user_scaled, data_scaled, cluster_label, kmeans, user_name, suffix=""):
    try:
        if user_scaled is None or cluster_label is None:
            raise ValueError("Invalid input data or cluster label for percentile calculation.")
        cluster_indices = np.where(kmeans.labels_ == cluster_label)[0]
        cluster_data = data_scaled[cluster_indices]
        centroid = kmeans.cluster_centers_[cluster_label]
        cluster_distances = pairwise_distances(cluster_data, [centroid], metric='euclidean').flatten()
        user_distance = pairwise_distances(user_scaled, [centroid], metric='euclidean')[0][0]
        percentile = 100 - (np.percentile(cluster_distances, np.where(cluster_distances <= user_distance)[0].size / len(cluster_distances) * 100) / np.percentile(cluster_distances, 100) * 100)

        # Visualize
        plt.figure(figsize=(10, 6))
        sns.histplot(cluster_distances, kde=True, color='blue', label='Cluster Distances')
        plt.axvline(user_distance, color='red', linestyle='--', label=f'{user_name} (Distance: {user_distance:.2f})')
        plt.title(f'Distance to Cluster {cluster_label} Centroid (Percentile: {percentile:.1f}%)')
        plt.xlabel('Distance to Centroid')
        plt.ylabel('Count')
        plt.legend()
        plt.savefig(f'cluster_percentile_plot{suffix}.png')
        plt.close()

        return percentile
    except Exception as e:
        print(f"Error in percentile calculation: {e}")
        return 0.0

# Prepare dataset for hypertension KMeans percentile
percentile_hypertension = 0.0
if kmeans_hypertension is not None and cluster_label_hypertension is not None:
    try:
        # Filter dataset by HE_HP
        data_hp = data[data['HE_HP'] == user_he_hp][features_hypertension_kmeans]
        if not data_hp.empty:
            data_hp_imputed = pd.DataFrame(imputer_hypertension_kmeans.transform(data_hp), columns=features_hypertension_kmeans)
            data_hp_scaled = scaler_hypertension_kmeans.transform(data_hp_imputed)
            # Fit KMeans to dataset to get labels
            data_cluster_labels = kmeans_hypertension.predict(data_hp_scaled)
            kmeans_hypertension.labels_ = data_cluster_labels
            percentile_hypertension = calculate_percentile_and_visualize(
                user_scaled_hypertension_kmeans, data_hp_scaled, cluster_label_hypertension,
                kmeans_hypertension, "사용자", f"_hypertension_HE_HP_{user_he_hp}"
            )
        else:
            print(f"No data available for HE_HP={user_he_hp} in dataset. Skipping percentile calculation.")
    except Exception as e:
        print(f"Error preparing hypertension dataset for percentile: {e}")

# --- Output Results ---
user_name = "사용자"
print("\n--- Health Prediction Results ---")
print(f"{user_name}님의 질병 예측 결과:")
for disease, result in disease_results.items():
    print(f"- {disease.capitalize()} ({'HE_DM_HbA1c' if disease == 'diabetes' else 'HE_HP' if disease == 'hypertension' else 'HE_obe'}): "
          f"'{result['stage']}'일 확률이 {result['probability']:.1f}%로 가장 높습니다.")

print(f"\n{user_name}님의 고혈압 클러스터 (HE_HP={user_he_hp}): {cluster_label_hypertension if cluster_label_hypertension is not None else 'N/A'}")
if cluster_label_hypertension is not None:
    print(cluster_description_hypertension)

print(f"\n{user_name}님은 고혈압 클러스터 (HE_HP={user_he_hp})에서 상위 {percentile_hypertension:.1f}%에 위치합니다.")

Diabetes model loaded (51 features).
Hypertension model loaded (56 features).
Obesity model loaded (28 features).
Hypertension KMeans model for HE_HP=2 loaded (19 features).

--- Health Prediction Results ---
사용자님의 질병 예측 결과:
- Diabetes (HE_DM_HbA1c): 'Diabetes'일 확률이 100.0%로 가장 높습니다.
- Hypertension (HE_HP): 'Stage 2 Hypertension'일 확률이 100.0%로 가장 높습니다.
- Obesity (HE_obe): 'Normal'일 확률이 100.0%로 가장 높습니다.

사용자님의 고혈압 클러스터 (HE_HP=2): 1
Cluster 1 (HE_HP=2): Elevated SBP, moderate metabolic risk.

사용자님은 고혈압 클러스터 (HE_HP=2)에서 상위 53.7%에 위치합니다.


군집에서 상위 몇 프로인지 그래프 추가

In [7]:
import pandas as pd
import numpy as np
import joblib
from sklearn.preprocessing import StandardScaler
from sklearn.impute import KNNImputer
from sklearn.metrics import pairwise_distances
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")

# --- Define Feature Lists (Fallbacks) ---
HE_DM_HbA1c_input_features = [
    'HE_glu', 'HE_HbA1c', 'DE1_dg', 'age', 'DI1_dg', 'HE_wc', 'DI2_dg', 'HE_sbp2', 'HE_sbp1', 'HE_BMI',
    'HE_BUN', 'HE_alt', 'HE_TG', 'HE_ast', 'L_OUT_FQ', 'HE_wt', 'D_1_1', 'HE_DMfh2', 'HE_dbp1', 'HE_dbp2',
    'BO1', 'HE_crea', 'sex', 'BP16_2', 'genertn', 'ho_incm', 'cfam', 'LS_VEG1', 'HE_chol', 'HE_LDL_drct',
    'L_BR_FQ', 'HE_Ucrea', 'HE_HDL_st2', 'edu', 'DI2_ag', 'DI2_pt', 'DI2_pr', 'DI2_2', 'DI1_ag', 'DI1_pt',
    'DI1_pr', 'DI1_2', 'DE1_33', 'DE1_31', 'DE1_32', 'DE1_34', 'DE1_ag', 'DE1_4', 'DE1_3', 'DE1_pt', 'DE1_pr'
]

HE_obe_input_features = [
    'HE_BMI', 'HE_wc', 'HE_wt', 'BO1', 'HE_alt', 'HE_TG', 'HE_sbp2', 'HE_sbp1', 'HE_glu', 'HE_dbp1',
    'HE_HbA1c', 'HE_dbp2', 'BO1_1', 'DI1_dg', 'HE_ast', 'HE_crea', 'DI2_dg', 'DI2_pt', 'DI2_pr', 'DI2_2',
    'sex', 'DI2_ag', 'DI1_pr', 'DI1_pt', 'DI1_2', 'DI1_ag', 'BO2_1', 'HE_HDL_st2'
]

HE_HP_input_features = [
    'DI1_dg', 'HE_sbp2', 'HE_sbp1', 'age', 'HE_dbp1', 'HE_dbp2', 'HE_wc', 'HE_HbA1c', 'HE_glu', 'DI2_dg',
    'HE_BMI', 'HE_ast', 'DE1_dg', 'HE_BUN', 'HE_TG', 'HE_alt', 'L_OUT_FQ', 'HE_wt', 'D_1_1', 'HE_HPfh2',
    'BD2_1', 'HE_crea', 'BD1_11', 'BP1', 'BO1', 'HE_ht', 'BD1', 'sex', 'HE_chol', 'BP16_2', 'HE_LDL_drct',
    'HE_HDL_st2', 'genertn', 'cfam', 'LS_VEG1', 'HE_Ucrea', 'DE1_ag', 'DE1_32', 'DE1_34', 'DE1_31', 'DE1_33',
    'L_BR_FQ', 'DE1_4', 'DE1_pt', 'DE1_pr', 'DE1_3', 'ho_incm', 'DI2_ag', 'DI2_pt', 'DI2_pr', 'DI2_2', 'edu',
    'DI1_ag', 'DI1_pt', 'DI1_pr', 'DI1_2'
]

kmeans_features = [
    'HE_sbp1', 'HE_dbp1', 'HE_sbp2', 'HE_dbp2', 'HE_wc', 'HE_wt', 'HE_ht', 'HE_BMI',
    'HE_glu', 'HE_HbA1c', 'HE_chol', 'HE_HDL_st2', 'HE_TG', 'HE_LDL_drct', 'HE_BUN',
    'HE_crea', 'HE_Ucrea', 'HE_ast', 'HE_alt', 'BD1_11', 'BP16_1', 'BS3_2', 'BE5_1',
    'L_OUT_FQ', 'LS_VEG1', 'LS_FRUIT'
]

# --- Load Models and Preprocessing Objects ---
def load_model_and_preprocessing(model_name, feature_list_fallback):
    try:
        model = joblib.load(f"{model_name}_decision_tree_model.pkl")
        imputer = joblib.load(f"{model_name}_imputer.pkl")
        scaler = joblib.load(f"{model_name}_scaler.pkl")
        try:
            features = joblib.load(f"{model_name}_features_list.pkl")
        except FileNotFoundError:
            print(f"Warning: {model_name}_features_list.pkl not found. Using fallback feature list.")
            features = feature_list_fallback
        print(f"{model_name.capitalize()} model loaded ({len(features)} features).")
        return model, imputer, scaler, features
    except FileNotFoundError as e:
        print(f"Error loading {model_name} model or preprocessing objects: {e}")
        exit()

# Load disease prediction models
model_diabetes, imputer_diabetes, scaler_diabetes, features_diabetes = load_model_and_preprocessing("diabetes", HE_DM_HbA1c_input_features)
model_hypertension, imputer_hypertension, scaler_hypertension, features_hypertension = load_model_and_preprocessing("hypertension", HE_HP_input_features)
model_obesity, imputer_obesity, scaler_obesity, features_obesity = load_model_and_preprocessing("obesity", HE_obe_input_features)

# --- Load Hypertension KMeans Models by HE_HP ---
def load_hypertension_kmeans(he_hp_value):
    try:
        kmeans = joblib.load(f"hypertension_kmeans_model_HE_HP_{he_hp_value}.pkl")
        imputer = joblib.load(f"hypertension_kmeans_imputer_HE_HP_{he_hp_value}.pkl")
        scaler = joblib.load(f"hypertension_kmeans_scaler_HE_HP_{he_hp_value}.pkl")
        features = joblib.load(f"hypertension_kmeans_features_list_HE_HP_{he_hp_value}.pkl")
        print(f"Hypertension KMeans model for HE_HP={he_hp_value} loaded ({len(features)} features).")
        return kmeans, imputer, scaler, features
    except FileNotFoundError as e:
        print(f"Error loading hypertension KMeans model for HE_HP={he_hp_value}: {e}")
        return None, None, None, None

# --- Load Dataset for Percentile Calculation ---
try:
    data = pd.read_csv("선택컬럼_건강데이터.csv")
except FileNotFoundError as e:
    print(f"Error loading dataset: {e}")
    exit()

# --- Cluster Descriptions (Customized for HE_HP) ---
cluster_descriptions_hypertension = {
    1: {  # HE_HP=1
        0: "Cluster 0 (HE_HP=1): Low blood pressure, moderate BMI, healthy metabolic profile.",
        1: "Cluster 1 (HE_HP=1): Slightly elevated SBP/DBP, higher TG levels.",
        2: "Cluster 2 (HE_HP=1): Moderate risk with elevated cholesterol and BMI."
    },
    2: {  # HE_HP=2
        0: "Cluster 0 (HE_HP=2): Pre-hypertension, high BMI, low HDL.",
        1: "Cluster 1 (HE_HP=2): Elevated SBP, moderate metabolic risk.",
        2: "Cluster 2 (HE_HP=2): High TG and cholesterol, active lifestyle.",
        3: "Cluster 3 (HE_HP=2): High risk with elevated liver markers."
    },
    3: {  # HE_HP=3
        0: "Cluster 0 (HE_HP=3): Stage 1 hypertension, high BMI, high TG.",
        1: "Cluster 1 (HE_HP=3): Elevated SBP/DBP, moderate cholesterol.",
        2: "Cluster 2 (HE_HP=3): High risk with metabolic syndrome traits."
    },
    4: {  # HE_HP=4
        0: "Cluster 0 (HE_HP=4): Stage 2 hypertension, very high BMI, low HDL.",
        1: "Cluster 1 (HE_HP=4): Severe SBP/DBP elevation, high cholesterol.",
        2: "Cluster 2 (HE_HP=4): High TG, elevated liver markers.",
        3: "Cluster 3 (HE_HP=4): Extreme risk with multiple comorbidities."
    }
}

# --- User Input ---
user_data = {
    'HE_glu': 110.5, 'HE_HbA1c': 6.1, 'DE1_dg': 0, 'age': 55, 'DI1_dg': 0, 'HE_wc': 90.2,
    'DI2_dg': 0, 'HE_sbp2': 135, 'HE_sbp1': 138, 'HE_BMI': 26.5, 'HE_BUN': 18.0, 'HE_alt': 30.0,
    'HE_TG': 160, 'HE_ast': 25.0, 'L_OUT_FQ': 3, 'HE_wt': 78.5, 'D_1_1': 0, 'HE_DMfh2': 0,
    'HE_dbp1': 85, 'HE_dbp2': 83, 'BO1': 1, 'HE_crea': 1.1, 'sex': 1, 'BP16_2': 5,
    'genertn': 4, 'ho_incm': 5, 'cfam': 3, 'LS_VEG1': 2, 'HE_chol': 220, 'HE_LDL_drct': 140,
    'L_BR_FQ': 1, 'HE_Ucrea': 80.0, 'HE_HDL_st2': 45.0, 'edu': 3, 'DI2_ag': 0, 'DI2_pt': 0,
    'DI2_pr': 0, 'DI2_2': 0, 'DI1_ag': 0, 'DI1_pt': 0, 'DI1_pr': 0, 'DI1_2': 0,
    'DE1_33': 0, 'DE1_31': 0, 'DE1_32': 0, 'DE1_34': 0, 'DE1_ag': 0, 'DE1_4': 0, 'DE1_3': 0,
    'DE1_pt': 0, 'DE1_pr': 0, 'HE_HPfh2': 0, 'BD2_1': 0, 'BD1_11': 2, 'BP1': 0, 'HE_ht': 170,
    'BD1': 0, 'BO1_1': 0, 'BO2_1': 0, 'BP16_1': 4, 'BS3_2': 10, 'BE5_1': 2, 'LS_FRUIT': 3,
    'HE_HP': 2  # 예시로 HE_HP=2 설정
}

# --- Determine HE_HP for KMeans ---
user_he_hp = user_data.get('HE_HP', 1)
if user_he_hp not in [1, 2, 3, 4]:
    print(f"Invalid HE_HP value: {user_he_hp}. Defaulting to HE_HP=1.")
    user_he_hp = 1

# Load hypertension KMeans model
kmeans_hypertension, imputer_hypertension_kmeans, scaler_hypertension_kmeans, features_hypertension_kmeans = load_hypertension_kmeans(user_he_hp)

# --- Process User Input ---
def process_user_input(user_data, features, imputer, scaler):
    try:
        user_df = pd.DataFrame([user_data]).reindex(columns=features)
        user_imputed = pd.DataFrame(imputer.transform(user_df), columns=features)
        user_scaled = scaler.transform(user_imputed)
        return user_df, user_scaled
    except Exception as e:
        print(f"Error processing user input: {e}")
        return None, None

# Process inputs for disease prediction
user_df_diabetes, user_scaled_diabetes = process_user_input(user_data, features_diabetes, imputer_diabetes, scaler_diabetes)
user_df_hypertension, user_scaled_hypertension = process_user_input(user_data, features_hypertension, imputer_hypertension, scaler_hypertension)
user_df_obesity, user_scaled_obesity = process_user_input(user_data, features_obesity, imputer_obesity, scaler_obesity)

# Process input for hypertension KMeans
user_df_hypertension_kmeans, user_scaled_hypertension_kmeans = process_user_input(user_data, features_hypertension_kmeans, imputer_hypertension_kmeans, scaler_hypertension_kmeans)

# --- Disease Prediction ---
def predict_disease_probabilities(user_scaled, model, stages):
    try:
        if user_scaled is None:
            raise ValueError("Invalid input data for prediction.")
        probs = model.predict_proba(user_scaled)[0]
        stage_index = np.argmax(probs)
        stage = stages.get(stage_index, "Unknown")
        prob = np.max(probs) * 100
        return {"stage": stage, "probability": prob}
    except Exception as e:
        print(f"Error in prediction: {e}")
        return {"stage": "Error", "probability": 0.0}

# Define stage mappings
diabetes_stages = {0: "Normal", 1: "Pre-diabetes", 2: "Diabetes"}
hypertension_stages = {0: "Normal", 1: "Pre-hypertension", 2: "Stage 1 Hypertension", 3: "Stage 2 Hypertension"}
obesity_stages = {0: "Normal", 1: "Overweight", 2: "Obese", 3: "Severely Obese"}

# Predict diseases
disease_results = {
    "diabetes": predict_disease_probabilities(user_scaled_diabetes, model_diabetes, diabetes_stages),
    "hypertension": predict_disease_probabilities(user_scaled_hypertension, model_hypertension, hypertension_stages),
    "obesity": predict_disease_probabilities(user_scaled_obesity, model_obesity, obesity_stages)
}

# --- Cluster Assignment ---
def assign_cluster(user_scaled, kmeans, cluster_descriptions, he_hp=None):
    try:
        if user_scaled is None:
            raise ValueError("Invalid input data for clustering.")
        cluster_label = kmeans.predict(user_scaled)[0]
        if he_hp is not None:
            cluster_description = cluster_descriptions.get(he_hp, {}).get(cluster_label, "No description available.")
        else:
            cluster_description = cluster_descriptions.get(cluster_label, "No description available.")
        return cluster_label, cluster_description
    except Exception as e:
        print(f"Error in cluster assignment: {e}")
        return None, "Cluster assignment failed."

# Hypertension KMeans cluster assignment
if kmeans_hypertension is not None:
    cluster_label_hypertension, cluster_description_hypertension = assign_cluster(
        user_scaled_hypertension_kmeans, kmeans_hypertension, cluster_descriptions_hypertension, user_he_hp
    )
else:
    cluster_label_hypertension, cluster_description_hypertension = None, "Hypertension KMeans model not loaded."

# --- Percentile Calculation and Visualization ---
def calculate_percentile_and_visualize(user_scaled, data_scaled, cluster_label, kmeans, user_name, suffix=""):
    try:
        if user_scaled is None or cluster_label is None:
            raise ValueError("Invalid input data or cluster label for percentile calculation.")
        cluster_indices = np.where(kmeans.labels_ == cluster_label)[0]
        cluster_data = data_scaled[cluster_indices]
        centroid = kmeans.cluster_centers_[cluster_label]
        cluster_distances = pairwise_distances(cluster_data, [centroid], metric='euclidean').flatten()
        user_distance = pairwise_distances(user_scaled, [centroid], metric='euclidean')[0][0]
        percentile = 100 - (np.percentile(cluster_distances, np.where(cluster_distances <= user_distance)[0].size / len(cluster_distances) * 100) / np.percentile(cluster_distances, 100) * 100)

        # Visualize with enhanced annotations
        plt.figure(figsize=(10, 6))
        sns.histplot(cluster_distances, kde=True, color='blue', label='Cluster Distances')
        plt.axvline(user_distance, color='red', linestyle='--', label=f'{user_name} (Distance: {user_distance:.2f})')

        # Add percentile annotation on the plot
        plt.text(
            user_distance, plt.ylim()[1] * 0.9,
            f'상위 {percentile:.1f}%',
            color='red', fontsize=12, ha='center',
            bbox=dict(facecolor='white', alpha=0.8, edgecolor='red')
        )

        # Customize title and labels
        plt.title(f'고혈압 클러스터 {cluster_label} 내 위치 (HE_HP={suffix.split("_HE_HP_")[-1] if "_HE_HP_" in suffix else "Unknown"}, 상위 {percentile:.1f}%)', fontsize=14)
        plt.xlabel('클러스터 중심까지의 거리', fontsize=12)
        plt.ylabel('데이터 포인트 수', fontsize=12)
        plt.legend()
        plt.grid(True, alpha=0.3)

        # Save the plot
        plot_filename = f'cluster_percentile_plot{suffix}.png'
        plt.savefig(plot_filename, bbox_inches='tight')
        plt.close()

        return percentile, plot_filename
    except Exception as e:
        print(f"Error in percentile calculation: {e}")
        return 0.0, None

# Prepare dataset for hypertension KMeans percentile
percentile_hypertension = 0.0
plot_filename_hypertension = None
if kmeans_hypertension is not None and cluster_label_hypertension is not None:
    try:
        # Filter dataset by HE_HP
        data_hp = data[data['HE_HP'] == user_he_hp][features_hypertension_kmeans]
        if not data_hp.empty:
            data_hp_imputed = pd.DataFrame(imputer_hypertension_kmeans.transform(data_hp), columns=features_hypertension_kmeans)
            data_hp_scaled = scaler_hypertension_kmeans.transform(data_hp_imputed)
            # Fit KMeans to dataset to get labels
            data_cluster_labels = kmeans_hypertension.predict(data_hp_scaled)
            kmeans_hypertension.labels_ = data_cluster_labels
            percentile_hypertension, plot_filename_hypertension = calculate_percentile_and_visualize(
                user_scaled_hypertension_kmeans, data_hp_scaled, cluster_label_hypertension,
                kmeans_hypertension, "사용자", f"_hypertension_HE_HP_{user_he_hp}"
            )
        else:
            print(f"No data available for HE_HP={user_he_hp} in dataset. Skipping percentile calculation.")
    except Exception as e:
        print(f"Error preparing hypertension dataset for percentile: {e}")

# --- Output Results ---
user_name = "사용자"
print("\n--- Health Prediction Results ---")
print(f"{user_name}님의 질병 예측 결과:")
for disease, result in disease_results.items():
    print(f"- {disease.capitalize()} ({'HE_DM_HbA1c' if disease == 'diabetes' else 'HE_HP' if disease == 'hypertension' else 'HE_obe'}): "
          f"'{result['stage']}'일 확률이 {result['probability']:.1f}%로 가장 높습니다.")

print(f"\n{user_name}님의 고혈압 클러스터 (HE_HP={user_he_hp}): {cluster_label_hypertension if cluster_label_hypertension is not None else 'N/A'}")
if cluster_label_hypertension is not None:
    print(cluster_description_hypertension)

print(f"\n{user_name}님은 고혈압 클러스터 (HE_HP={user_he_hp})에서 상위 {percentile_hypertension:.1f}%에 위치합니다.")
if plot_filename_hypertension:
    print(f"자세한 그래프는 '{plot_filename_hypertension}'에서 확인하세요.") #사진 파일로 저장됨
else:
    print("퍼센타일 그래프를 생성하지 못했습니다.")

Diabetes model loaded (51 features).
Hypertension model loaded (56 features).
Obesity model loaded (28 features).
Hypertension KMeans model for HE_HP=2 loaded (19 features).

--- Health Prediction Results ---
사용자님의 질병 예측 결과:
- Diabetes (HE_DM_HbA1c): 'Diabetes'일 확률이 100.0%로 가장 높습니다.
- Hypertension (HE_HP): 'Stage 2 Hypertension'일 확률이 100.0%로 가장 높습니다.
- Obesity (HE_obe): 'Normal'일 확률이 100.0%로 가장 높습니다.

사용자님의 고혈압 클러스터 (HE_HP=2): 1
Cluster 1 (HE_HP=2): Elevated SBP, moderate metabolic risk.

사용자님은 고혈압 클러스터 (HE_HP=2)에서 상위 53.7%에 위치합니다.
자세한 그래프는 'cluster_percentile_plot_hypertension_HE_HP_2.png'에서 확인하세요.
